# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [324]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)

def remove_note(value):
    my_str=re.search('.*(?=\[note \d{1,2}\])',value)
    if my_str:
        return my_str.group()
    return value

def remove_star(value):
    my_str = re.search(r'(.*)(\*)',value)
    if my_str:
        return my_str.group(1)
    return value

def area_team(value):
    my_str = re.search(r'([A-Za-z.]*) (.*)', value)
    return my_str.group(1)

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,8]]
cities = cities.applymap(lambda x:remove_note(x)).replace(['—', '— ', ''],np.nan)
cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype('float')
cities.set_index('Metropolitan area',inplace=True)
cities.sort_index(inplace=True)

nhl_df=pd.read_csv("assets/nhl.csv")
nhl_df = nhl_df[nhl_df['year'] == 2018][['team', 'W', 'L']]
nhl_df = nhl_df[~((nhl_df['W'].str.contains(r'.*Division'))&(nhl_df['L'].str.contains(r'.*Division')))]
nhl_df['team'] = nhl_df['team'].apply(lambda x:remove_star(x))
# nhl_df = nhl_df.reset_index().iloc[:,[1,2,3]]
nhl_df.reset_index(drop=True,inplace=True)
nhl_df['WL_Ratio'] = nhl_df['W'].astype(int) / (nhl_df['W'].astype(int) + nhl_df['L'].astype(int))
nhl_df['Area'] = nhl_df['team'].apply(lambda x:area_team(x))
nhl_df['Area'][0] = 'Tampa Bay Area'
nhl_df['Area'][3] = 'Miami–Fort Lauderdale'
nhl_df['Area'][8] = 'Washington, D.C.'
nhl_df['Area'][12] = 'New York City'
nhl_df['Area'][13] = 'Raleigh'
nhl_df['Area'][14] = 'New York City'
nhl_df['Area'][15] = 'New York City'
nhl_df['Area'][18] = 'Minneapolis–Saint Paul'
nhl_df['Area'][19] = 'Denver'
nhl_df['Area'][20] = 'St. Louis'
nhl_df['Area'][21] = 'Dallas–Fort Worth'
nhl_df['Area'][23] = 'Las Vegas'
nhl_df['Area'][24] = 'Los Angeles'
nhl_df['Area'][25] = 'San Francisco Bay Area'
nhl_df['Area'][26] = 'Los Angeles'
nhl_df['Area'][30] = 'Phoenix'
nhl_df.set_index('Area', inplace=True)
nhl_df = nhl_df.groupby(nhl_df.index).mean()

def nhl_correlation(): 
    # YOUR CODE HERE
    final = nhl_df.join(cities)
    # raise NotImplementedError()
    
    population_by_region = list(final['Population (2016 est.)[8]']) # pass in metropolitan area population from cities
    win_loss_by_region = list(final['WL_Ratio']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [325]:
nhl_correlation()

0.012486162921209881

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [326]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)

def get_region(value):
    my_str = re.search(r'(\w*) (.*)',value)
    return my_str.group(1)

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,7]]
cities = cities.applymap(lambda x:remove_note(x)).replace(['—', '— ', ''],np.nan)
cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype('float')
cities.set_index('Metropolitan area',inplace=True)
cities.sort_index(inplace=True)

nba_df=pd.read_csv("assets/nba.csv")
nba_df = nba_df[nba_df['year']==2018][['team', 'W', 'L']]
nba_df['Area'] = nba_df['team'].apply(lambda x:get_region(x))
nba_df['WL_Ratio'] = nba_df['W'].astype(int) / (nba_df['W'].astype(int) + nba_df['L'].astype(int))
nba_df['Area'][4] = 'Indianapolis'
nba_df['Area'][5] = 'Miami–Fort Lauderdale'
nba_df['Area'][7] = 'Washington, D.C.'
nba_df['Area'][10] = 'New York City'
nba_df['Area'][11] = 'New York City'
nba_df['Area'][16] = 'San Francisco Bay Area'
nba_df['Area'][18] = 'Oklahoma City'
nba_df['Area'][19] = 'Salt Lake City'
nba_df['Area'][20] = 'New Orleans'
nba_df['Area'][21] = 'San Antonio'
nba_df['Area'][22] = 'Minneapolis–Saint Paul'
nba_df['Area'][24] = 'Los Angeles'
nba_df['Area'][25] = 'Los Angeles'
nba_df['Area'][27] = 'Dallas–Fort Worth'
nba_df = nba_df[['Area','WL_Ratio']]
nba_df.set_index('Area', inplace=True)
nba_df = nba_df.groupby(nba_df.index).mean()

def nba_correlation():
    # YOUR CODE HERE
    final = nba_df.join(cities)
    # raise NotImplementedError()
    
    population_by_region = list(final['Population (2016 est.)[8]']) # pass in metropolitan area population from cities
    win_loss_by_region = list(final['WL_Ratio']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [327]:
nba_correlation()

-0.17657160252844611

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [328]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)

def get_area(value):
    my_str = re.search(r'([\w.]*) (.*)',value)
    return my_str.group(1)

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,6]]
cities = cities.applymap(lambda x:remove_note(x)).replace(['—', '— ', ''],np.nan)
cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype('float')
cities.set_index('Metropolitan area',inplace=True)
cities.sort_index(inplace=True)

mlb_df=pd.read_csv("assets/mlb.csv")
mlb_df = mlb_df[mlb_df['year'] == 2018][['team', 'W', 'L']]
mlb_df['Area'] = mlb_df['team'].apply(lambda x:get_area(x))
mlb_df['WL_Ratio'] = mlb_df['W'].astype(int) / (mlb_df['W'].astype(int) + mlb_df['L'].astype(int))
mlb_df['Area'][1] = 'New York City'
mlb_df['Area'][2] = 'Tampa Bay Area'
mlb_df['Area'][6] = 'Minneapolis–Saint Paul'
mlb_df['Area'][9] = 'Kansas City'
mlb_df['Area'][11] = 'San Francisco Bay Area'
mlb_df['Area'][13] = 'Los Angeles'
mlb_df['Area'][14] = 'Dallas–Fort Worth'
mlb_df['Area'][16] = 'Washington, D.C.'
mlb_df['Area'][18] = 'New York City'
mlb_df['Area'][19] = 'Miami–Fort Lauderdale'
mlb_df['Area'][22] = 'St. Louis'
mlb_df['Area'][25] = 'Los Angeles'
mlb_df['Area'][26] = 'Denver'
mlb_df['Area'][27] = 'Phoenix'
mlb_df['Area'][28] = 'San Francisco Bay Area'
mlb_df['Area'][29] = 'San Diego'
mlb_df = mlb_df[['Area','WL_Ratio']]
mlb_df.set_index('Area', inplace=True)
mlb_df = mlb_df.groupby(mlb_df.index).mean()

def mlb_correlation(): 
    # YOUR CODE HERE
    final = mlb_df.join(cities)
    # raise NotImplementedError()
    
    population_by_region = list(final['Population (2016 est.)[8]']) # pass in metropolitan area population from cities
    win_loss_by_region = list(final['WL_Ratio']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [329]:
mlb_correlation()

0.1502769830266931

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [330]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)

def get_area(value):
    my_str = re.search(r'([\w.]*) (.*)',value)
    return my_str.group(1)

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5]]
cities = cities.applymap(lambda x:remove_note(x)).replace(['—', '— ', ''],np.nan)
cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype('float')
cities.set_index('Metropolitan area',inplace=True)
cities.sort_index(inplace=True)

nfl_df=pd.read_csv("assets/nfl.csv")
nfl_df = nfl_df[nfl_df['year'] == 2018][['team', 'W', 'L']]
nfl_df['Area'] = nfl_df['team'].apply(lambda x:get_area(x))
nfl_df = nfl_df[~((nfl_df['W'].str.contains(r'AFC*')) | (nfl_df['W'].str.contains(r'NFC*')))]
nfl_df['WL_Ratio'] = nfl_df['W'].astype(int) / (nfl_df['W'].astype(int) + nfl_df['L'].astype(int))
nfl_df.reset_index(drop=True,inplace=True)
nfl_df['Area'][0] = 'Boston'
nfl_df['Area'][1] = 'Miami–Fort Lauderdale'
nfl_df['Area'][3] = 'New York City'
nfl_df['Area'][10] = 'Nashville'
nfl_df['Area'][12] = 'Kansas City'
nfl_df['Area'][13] = 'Los Angeles'
nfl_df['Area'][15] = 'San Francisco Bay Area'
nfl_df['Area'][16] = 'Dallas–Fort Worth'
nfl_df['Area'][18] = 'Washington, D.C.'
nfl_df['Area'][19] = 'New York City'
nfl_df['Area'][21] = 'Minneapolis–Saint Paul'
nfl_df['Area'][22] = 'Green Bay'
nfl_df['Area'][24] = 'New Orleans'
nfl_df['Area'][25] = 'Charlotte'
nfl_df['Area'][27] = 'Tampa Bay Area'
nfl_df['Area'][28] = 'Los Angeles'
nfl_df['Area'][30] = 'San Francisco Bay Area'
nfl_df['Area'][31] = 'Phoenix'
nfl_df = nfl_df[['Area','WL_Ratio']]
nfl_df.set_index('Area', inplace=True)
nfl_df = nfl_df.groupby(nfl_df.index).mean()

def nfl_correlation(): 
    # YOUR CODE HERE
    final = nfl_df.join(cities)
    # raise NotImplementedError()
    
    population_by_region = list(final['Population (2016 est.)[8]']) # pass in metropolitan area population from cities
    win_loss_by_region = list(final['WL_Ratio']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [331]:
nfl_correlation()

0.004922112149349456

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [322]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    # raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [323]:
cities

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards[note 11],Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]
7,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild[note 16]
9,Denver,2853077,Broncos,Rockies,Nuggets[note 17],Avalanche[note 18]
